In [1]:
import pandas as pd
from glob import glob
import numpy as np
import seaborn as sns
import re

In [22]:
files=glob(r'C:\Users\Jeronimo.Pardo\Documents\lll\Noggin new\*.csv')

In [46]:
br_post= [x for x in files if re.search(r'.*\(Post Level\) - Noggin BR*', x)]
la_post= [x for x in files if re.search(r'.*\(Post Level\) - Noggin La*', x)]

## FB Post LA

In [50]:
fb_post_la= pd.read_csv(la_post[0],sep=',', skiprows=[1], index_col=False)

In [52]:
## Fechas

new = fb_post_la['Posted'].str.split(" ", n = 1, expand = True) 
fb_post_la['Fecha'] = new[0]
fb_post_la['Hora'] = new[1]
fb_post_la['Dia'] = pd.DatetimeIndex(fb_post_la['Fecha']).dayofweek
dayDict = {6:'Domingo', 0:'Lunes', 1:'Martes', 2:'Miércoles', 3:'Jueves', 4: 'Viernes', 5:'Sábado' }
fb_post_la['Dia']=fb_post_la['Dia'].map(dayDict)

fb_post_la['Fecha']= pd.to_datetime(fb_post_la['Fecha']) 
fb_post_la['Mes'] = fb_post_la.Fecha.dt.month
fb_post_la['Hora']= pd.to_datetime(fb_post_la['Hora']) 
fb_post_la['Hora'] =  fb_post_la.Hora.dt.hour


##Campos calculados
fb_post_la=fb_post_la.fillna(0)
fb_post_la['VTR'] = fb_post_la['Lifetime Organic Video Views'] / fb_post_la['Lifetime Post Total Reach']

fb_post_la['E.R.'] =(fb_post_la['Lifetime Talking About This (Post) by action type - comment'] + fb_post_la['Lifetime Talking About This (Post) by action type - like'] + fb_post_la['Lifetime Talking About This (Post) by action type - share']+ fb_post_la['Lifetime Post Audience Targeting Unique Consumptions by Type - other clicks'] + fb_post_la['Lifetime Post Audience Targeting Unique Consumptions by Type - link clicks']) / fb_post_la['Lifetime Post Total Reach']
fb_post_la=fb_post_la.replace([np.inf, -np.inf], np.nan)

fb_post_la=fb_post_la.fillna(0)


In [54]:
fb_post_br= pd.read_csv(br_post[0],sep=',', skiprows=[1], index_col=False)

## IG Post LA

In [12]:
ig_post_la= pd.read_excel(open('rolling-export-nogginla-936364-1125393-20190201-20190630-1564153006.xlsx', 'rb'),
...               sheet_name='My media', skiprows=[0])

In [13]:
ig_post_la=ig_post_la.drop(['Caption'], axis=1)

new2 = ig_post_la['Date and time'].str.split(" ", n = 1, expand = True) 
ig_post_la['Fecha'] = new2[0]
ig_post_la['Hora'] = new2[1]
ig_post_la['Dia'] = pd.DatetimeIndex(ig_post_la['Fecha']).dayofweek
dayDict = {6:'Domingo', 0:'Lunes', 1:'Martes', 2:'Miércoles', 3:'Jueves', 4: 'Viernes', 5:'Sábado' }
ig_post_la['Dia']=ig_post_la['Dia'].map(dayDict)

ig_post_la['Fecha']= pd.to_datetime(ig_post_la['Fecha']) 
ig_post_la['Mes'] = ig_post_la.Fecha.dt.month
ig_post_la['Hora']= pd.to_datetime(ig_post_la['Hora']) 
ig_post_la['Hora'] =  ig_post_la.Hora.dt.hour


ig_post_la=ig_post_la.fillna(0)

ig_post_la['VTR'] =  ig_post_la['Views'] / ig_post_la['Reach']

In [33]:
ig_post_la.Views.sum()

2469.0

## Evolución LA

In [87]:
f= fb_post_la.groupby('Mes')['Lifetime Post Total Reach','Lifetime Paid Video Views'].sum().reset_index()

i = ig_post_la.groupby('Mes')['Reach', 'Views'].sum().reset_index()

fi= pd.merge(f,i)

fi['Views Total'] = fi['Lifetime Paid Video Views'] + fi['Views']

fi['Views Total'] = fi['Views Total'].apply(lambda x: int(x))

fi.drop('Views', axis=1, inplace=True)
fi['Alcance Total'] = fi['Lifetime Post Total Reach'] + fi['Reach']

In [88]:
mesDict = {2:'Febrero', 3:'Marzo', 4:'Abril', 5:'Mayo', 6:'Junio', 7: 'Julio', 8:'Agosto', 9:'Septiembre', 10:'Octubre', 11:'Noviembre', 12:'Diciembre' }
fi['Mes2']=fi['Mes'].map(mesDict)

In [89]:
fi = fi[['Mes2', 'Mes','Alcance Total', 'Views Total']]

In [90]:
fi.to_csv('EvolLA.csv', index=False)